In [ ]:
import pandas as pd
import folium # pip install folium
import seaborn as sns # pip install seaborn
import matplotlib.pyplot as plt
import plotly.express as px # pip install plotly

from collections import Counter

plt.rc('font', family='Malgun Gothic') # 한글 폰트 설정 (Malgun Gothic)
plt.rc('axes', unicode_minus=False) # 마이너스 기호 깨짐 방지

# 공용 변수 선언
dataIn = './../dataIn/'
dataOut = './../dataOut/'

In [ ]:
# 이미 생성해 놓은 매장 정보 파일을 읽어 옵니다.
starbucks = pd.read_csv(dataOut + 'starbucks_file.csv')
ediya = pd.read_csv(dataOut + 'ediya_file.csv')
hollysBefore = pd.read_csv(dataOut + 'hollys_file.csv')

In [ ]:
print(f'스타벅스 매장 갯수 : {len(starbucks)}')
print(f'이디야 매장 갯수 : {len(ediya)}')
print(f'할리스 매장 갯수 : {len(hollysBefore)}')

In [ ]:
starbucks.columns

In [ ]:
ediya.columns

In [ ]:
hollysBefore.columns

In [ ]:
print('스타벅스와 할리스의 공통 컬럼 정보')
common_col = starbucks.columns.intersection(hollysBefore.columns)
common_col

In [ ]:
print('# 다른 매장들과 공통된 컬럼들만 따로 추출')
hollys = hollysBefore[common_col]
hollys.columns

In [ ]:
print('# 각 매장의 데이터 프레임을 하나로 모아줍니다')
coffeeFrame = pd.concat([starbucks, ediya, hollys], axis=0)
print(f'전체 매장 갯수 : {len(coffeeFrame)}')

In [ ]:
filename = dataOut + 'coffee_list.csv'
coffeeFrame.to_csv(filename, index=False, encoding='UTF-8')
print(f'{filename} 파일이 저장되었습니다.')

In [ ]:
coffeeFrame.info()

In [ ]:
print(f'위도/경도 결측지 제거전 : {len(coffeeFrame)}')

In [ ]:
# notnull() 함수는 결측치가 아니면 True를 반환해주는 함수 ↔ isnull()
mapFrame = coffeeFrame[coffeeFrame['위도'].notnull()]

print(f'위도/경도 결측지 제거후 : {len(mapFrame)}')

In [ ]:
# 지도 위에 매장 표시
# folium에서 사용할 브랜드별 색상 지정
brand_colors = {
    '스타벅스': 'beige',
    '이디야': 'darkpurple',
    '할리스': 'lightblue'
}

seoul_center = [mapFrame['위도'].mean(), mapFrame['경도'].mean()] # 서울 중심 위치
seoul_center

In [ ]:
# 서울 지도 객체
seoul_map = folium.Map(location=seoul_center, zoom_start=11)

# iterrows() : iterable rows(전체 행을 반복)
# for 색인_번호, 데이터_1행 in mapFrame.iterrows():
# 찾아 볼것 : 마커 크기 조정, popup의 모양
for _, row in mapFrame.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['상호']}({row['브랜드']})", # 팝업 예시 : xxx점(스타벅스)
        icon=folium.Icon(color=brand_colors.get(row['브랜드'], 'gray'))
    ).add_to(seoul_map)
# end for

htmlfile = dataOut + 'coffee_map.html'
seoul_map.save(htmlfile)
print(f'{htmlfile} 파일 생성')

In [ ]:
seoul_map

In [ ]:
print('# 군구 컬럼이 마포구, 용산구, 서대문구인 항목들만 추출')
concern_gu = ['마포구', '용산구', '서대문구']
myfilter01 = mapFrame[mapFrame['군구'].isin(concern_gu)]
print(f'데이터 갯수 : {len(myfilter01)}')

In [ ]:
myfilter01.head()

In [ ]:
my_center = [myfilter01['위도'].mean(), myfilter01['경도'].mean()]

filter_map = folium.Map(location=my_center, zoom_start=13, tiles='OpenStreetMap')

for _, row in myfilter01.iterrows():
    # 브랜드별로 색상을 다르게 지정하려면 brand_colors에 추가하는 방법도 고려해 보세요.
    ic_color = 'white' # 아이콘 색상

    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['주소'],
        tooltip=f"{row['상호']}({row['브랜드']})",
        icon=folium.Icon(
            color=brand_colors.get(row['브랜드'], 'gray'),
            icon_color=ic_color,
            icon='coffee',
            prefix='fa'
        )
    ).add_to(filter_map)
# end for

htmlfile = dataOut + 'coffee_map_02.html'
seoul_map.save(htmlfile)
print(f'{htmlfile} 파일 생성')

In [ ]:
filter_map

In [ ]:
# Plotly로 인터랙티브 지도 시각화
fig = px.scatter_map(
    mapFrame, lat='위도', lon='경도', color='브랜드', hover_name='상호', zoom=11, map_style='open-street-map',
)
fig.show()

In [ ]:
coffeeFrame.head()

In [ ]:
# 챠트 그리기 시작
mycolor = ['green', 'blue', 'red']

In [ ]:
# value_counts() 함수는 빈도 수가 큰 것부터 역순으로 정렬해 줍니다.
chart_data = coffeeFrame['브랜드'].value_counts()
chart_data.to_frame()

In [ ]:
plt.figure(figsize=(6, 6)) # 도화지 준비

plt.title('브랜드별 매장 비율')
chart_data.plot(kind='pie', autopct='%.1f%%', colors=mycolor, shadow=True, explode=(0, 0.1, 0))
plt.ylabel('') # y축의 라벨 제거
plt.savefig(dataOut + 'coffee_01.png')

In [ ]:
# 계수기(Counter) 사용
brand_counts = Counter(coffeeFrame['브랜드'])
brand_counts

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(brand_counts.keys(), brand_counts.values(), color=mycolor)

plt.title('브랜드별 매장 갯수', size=18)
plt.xlabel('브랜드', size=13)
plt.ylabel('매장 갯수', size=13)
plt.xticks(rotation=30)
plt.savefig(dataOut + 'coffee_02.png')

In [ ]:
plt.figure(figsize=(10, 6))

# myorder = sorted(coffeeFrame['군구'].value_counts().index)
myorder = coffeeFrame['군구'].value_counts().index
# myorder

# countplot 함수 : 범주형 데이터의 빈도를 막대 그래프로 보여 줍니다.
sns.countplot(data=coffeeFrame, x='군구', order=myorder, hue='군구', palette='viridis')

plt.title('구별 매장 갯수', size=18)
plt.xlabel('군구', size=13)
plt.ylabel('매장 갯수', size=13)
plt.xticks(rotation=30)
plt.savefig(dataOut + 'coffee_03.png')

In [ ]:
plt.figure(figsize=(10, 8))

# scatterplot 함수는 두 개의 연속형 데이터 사이의 관계를 점으로 그려 주는 그래프
sns.scatterplot(data=mapFrame, x='위도', y='경도', hue='브랜드')

plt.title('브랜드별 매장 위치 분포', size=18)
plt.xlabel('위도', size=13)
plt.ylabel('경도', size=13)
plt.savefig(dataOut + 'coffee_04.png')

In [ ]:
# 구별, 브랜드별 점유율
gu_brand_data = coffeeFrame.groupby(['군구', '브랜드'])
gu_brand_data

In [ ]:
# 중첩된 색인을 단일 색인으로 변경하고, 누락이 된 데이터는 0으로 채워주세요.
# unstack() 함수는 중첩된 색인을, 2차원 형식의 표(pivot) 형태로 변형시켜 줍니다.
gu_brand_chart = gu_brand_data.size().unstack(fill_value=0)
gu_brand_chart

In [ ]:
plt.figure(figsize=(12, 8))
gu_brand_chart.plot(kind='barh', colormap='viridis')

plt.title('구별 브랜드별 매장 갯수', size=18)
plt.ylabel('군구', size=13)
plt.xlabel('매장 갯수', size=13)
plt.xticks(rotation=30)
plt.legend(title='브랜드')
plt.savefig(dataOut + 'coffee_05.png')

In [ ]:
plt.figure(figsize=(12, 8))
gu_brand_chart.plot(kind='barh', colormap='viridis', stacked=True)

plt.title('구별 브랜드별 매장 갯수(누적)', size=18)
plt.ylabel('군구', size=13)
plt.xlabel('매장 갯수', size=13)
plt.xticks(rotation=30)
plt.legend(title='브랜드')
plt.savefig(dataOut + 'coffee_06.png')

In [ ]:
# 할리스 편의 서비스 정보
# 1. 편의 서비스를 하나도 제공하지 않는 매장 정보
# 2. 2개 이상의 서비스를 제공하는 매장 정보
# 3. 2개 이상의 서비스를 제공하는 곳 중에서 가장 많은 구 정보

In [ ]:
# 편의 서비스 관련 컬럼 리스트
service_columns = ['상호', '24시간', 'DT 매장', '주차', '테라스', '흡연시설']
hollysBefore[service_columns].head()

In [ ]:
# 파생 컬럼 추가 : '서비스 제공 개수'에는 'yes' 개수를 카운터하여 저장합니다.
hollysBefore['서비스 제공 개수'] = hollysBefore[service_columns].apply(lambda row: sum(row == 'yes'), axis=1)
hollysBefore.head()

In [ ]:
print('# 편의 서비스 제공을 하나도 제공하지 않는 매장')
no_service_stores = hollysBefore[hollysBefore['서비스 제공 개수'] == 0]
# no_service_stores[service_columns].head()

filename = dataOut + 'hollys_no_service_stores.csv'
no_service_stores.to_csv(filename, index=False, encoding='UTF-8')
print(f'{filename} 파일이 저장되었습니다.')

In [ ]:
print('# 2개 이상의 편의 서비스 제공을 하는 매장')
multiple_service_stores = hollysBefore[hollysBefore['서비스 제공 개수'] >= 2]

filename = dataOut + 'hollys_multiple_service_stores.csv'
multiple_service_stores.to_csv(filename, index=False, encoding='UTF-8')
print(f'매장 갯수 : {len(multiple_service_stores)}개')
print(f'{filename} 파일이 저장되었습니다.')

In [ ]:
# '군구'별 '서비스 제공 개수'의 그룹핑 객체
condition = multiple_service_stores.groupby('군구')['서비스 제공 개수']
type(condition)

In [ ]:
# idxmax() 함수는 값이 가장 큰 항목의 색인 정보를 반환합니다. 반대 함수는 idxmin()
print(f'구별 서비스 제공 갯수의 총합이 가장 큰 구 : {condition.sum().idxmax()}')

In [ ]:
top05 = condition.sum().sort_values(ascending=False).head(n=5)
top05 # 필요하다면 그래프 그리기

In [105]:
# mode() 함수는 최빈도를 구해 주는 함수입니다.
most_common_gu =  multiple_service_stores['군구'].mode()[0]
print(f'2개 이상의 서비스 제공의 총합이 가장 많은 구 : {most_common_gu}')

2개 이상의 서비스 제공의 총합이 가장 많은 구 : 동작구


In [108]:
most_common_gu_stores = multiple_service_stores[multiple_service_stores['군구'] == most_common_gu]
print(f'{most_common_gu}의 매장 정보 확인')
most_common_gu_stores

동작구의 매장 정보 확인


,브랜드,상호,주소,시도,군구,위도,경도,전화 번호,24시간,DT 매장,주차,테라스,흡연시설,서비스 제공 개수
47,할리스,중앙대점,서울특별시 동작구 흑석로 77 (흑석동) 1~4층,서울특별시,동작구,37.506983,126.958246,02-3280-3280,no,no,no,yes,yes,2
58,할리스,대방역점,"서울특별시 동작구 알마타길 6 (대방동, 파밀리에 하늘마루) 101호, 102호",서울특별시,동작구,37.509857,126.924954,02-825-6369,no,no,yes,yes,no,2
93,할리스,노량진역점,서울특별시 동작구 노량진로 157 2-5층,서울특별시,동작구,37.513979,126.943356,02-827-0280,yes,no,no,no,yes,2
100,할리스,이수역점,"서울특별시 동작구 동작대로 83 (사당동, 헤라피스빌딩) B1~1층",서울특별시,동작구,37.484012,126.981551,02-588-4845,no,no,yes,no,yes,2


In [109]:
most_common_gu_center = [most_common_gu_stores['위도'].mean(), most_common_gu_stores['경도'].mean()]

most_common_gu_map = folium.Map(location=most_common_gu_center, zoom_start=13, tiles='OpenStreetMap')

for _, row in most_common_gu_stores.iterrows():
    # 브랜드별로 색상을 다르게 지정하려면 brand_colors에 추가하는 방법도 고려해 보세요.
    ic_color = 'white' # 아이콘 색상

    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['주소'],
        tooltip=f"{row['상호']}({row['브랜드']})",
        icon=folium.Icon(
            color='blue',
            icon_color=ic_color,
            icon='info-sign',
            prefix='glyphicon'
        )
    ).add_to(most_common_gu_map)
# end for

htmlfile = dataOut + 'coffee_map_03.html'
most_common_gu_map.save(htmlfile)
print(f'{htmlfile} 파일 생성')

./../dataOut/coffee_map_03.html 파일 생성


In [110]:
most_common_gu_map

In [2]:
import pandas as pd
import requests
import time
import re
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

In [3]:
# 크롬 드라이버 다운로드 :  https://googlechromelabs.github.io/chrome-for-testing/
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간 10초
driver.implicitly_wait(wait_time)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 141
Current browser version is 143.0.7499.41 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
	GetHandleVerifier [0x0x7ff698d8e9e5+80021]
	GetHandleVerifier [0x0x7ff698d8ea40+80112]
	(No symbol) [0x0x7ff698b1060f]
	(No symbol) [0x0x7ff698b55f99]
	(No symbol) [0x0x7ff698b54d6c]
	(No symbol) [0x0x7ff698b4eca4]
	(No symbol) [0x0x7ff698b4a2d7]
	(No symbol) [0x0x7ff698b9f040]
	(No symbol) [0x0x7ff698b9e7b0]
	(No symbol) [0x0x7ff698b91003]
	(No symbol) [0x0x7ff698b595d1]
	(No symbol) [0x0x7ff698b5a3f3]
	GetHandleVerifier [0x0x7ff69904dd8d+2960445]
	GetHandleVerifier [0x0x7ff69904804a+2936570]
	GetHandleVerifier [0x0x7ff699068a87+3070263]
	GetHandleVerifier [0x0x7ff698da84ce+185214]
	GetHandleVerifier